In [2]:
import torch 
import pandas as pd
from torch.utils.data import random_split,DataLoader,TensorDataset

## Problema 1

In [21]:
dataset = pd.read_csv("dataset.csv")
dataset.drop("#",axis=1,inplace=True)
dataset

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


### Descripción del dataset  
En el siguiente bloque describa el dataset a utilizar. También es necesario hacer un análisis exploratorio de los datos.
Responda como mínimo las siguientes preguntas:
- ¿De qué trata el dataset?
- ¿Cuáles son las clases de salida?
- ¿Está preparado para poder ser introducido a una red neuronal?
- ¿Cuantas carácteristicas tiene?
- Etc.


#### Dataset: Pokemon
Los datos que estamos viendo en este dataset son las distintas estadisticas que poseen los pokemon de 1era a 6ta generacion, mostrando su nombre y sus tipos correspondientes, ademas de especificar si este pokemon es legendario o no

Las caracteristicas que posee el dataset tiene los siguientes valores:
* **Name:** Nombre del Pokemon en cuestion
* **Type 1:** Primer tipo del pokemon, que siempre contiene un valor (Fire, Fairy, Steel, Water, Grass, Dragon, Ice, Poison, Bug, Normal, Flying, Fighting, Ghost, Psychic, Dark, Ground, Rock y Electric)
* **Type 2:** Tipo secundario del pokemon, que puede no tener.
* **HP:** (Health Points) Cantidad de vida que el pokemon posee
* **Attack:** Valor base que tienen los pokemon para el ataque fisico
* **Defense:** Valor base de defensa que tienen los pokemon para resistir ataques fisicos
* **Sp. Atk:** Valor base de ataque especial que tienen los pokemon para inflingir daño especial
* **Sp. Def:** Valor base de defensa especial que tienen los pokemon para resistir ataques especiales
* **Speed:** Valor base de un pokemon para definir cual de los pokemones se mueve primero en batalla
* **Total:** Suma total de todas las estadisticas de un pokemon, donde normalmente los que son sobre 600 son legendarios
* **Generation:** Generacion a la cual pertenece el pokemon, refiriendonos a los juegos que salieron el mismo año (Ej: Rojo, Azul y Amarillo salieron el mismo año y se consideran 1era generacion)
* **Legendary:** Valor que define si el pokemon en cuestion es legendario o no (True: Legendario / False: No legendario)

Dependiendo de cual sea el estudio, el valor de salida pueden ser distintas, como por ejemplo, los tipos de el pokemon con el que normalmente se le puede asociar una estadistica, como por ejemplo el tipo acero tiene principalmente tipo acero, asi mismo, si veo un pokemon con una stat mayor en la defensa, lo mas probable es que sea acero, o viendo las estadisticas totales, puedo analizar si el pokemon en cuestion es o no legendario

El Dataset de por si no posee un valor que tenga 0 en alguna parte, ya que no puede existir una estadistica en 0, tampoco los nombres pueden ser vacio, el unico valor que puede llegar a existir que sea un valor no leible seria el NaN en el segundo tipo, que esto implica que el pokemon solo tiene un tipo.

In [5]:
# Verificar valores nulos en otras columnas
columnas = ["Total", "HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Generation", "Legendary"]
valores_nulos = dataset[columnas].isnull().sum()

# Mostrar los resultados
print(valores_nulos)

valores_cero = dataset[columnas].eq(0).sum()

# Mostrar los resultados
print(valores_cero)
# En legendary es normal ya que es verdadero o falso, no un valor numerico

Total         0
HP            0
Attack        0
Defense       0
Sp. Atk       0
Sp. Def       0
Speed         0
Generation    0
Legendary     0
dtype: int64
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary     735
dtype: int64


#### Analisis exploratorio

### Preparación del dataset  
En el siguiente bloque realice la preparación del dataset. Esto incluye:
- Paso a tensores de PyTorch
- División del dataset en train y test
- Carga de los datos en DataLoader
- Otros pasos que considere necesarios

In [22]:
""" PREPARANDO EL DATASET """
# Eliminar columnas innecesarias
dataset.drop("Name", axis=1, inplace=True)

# Aplicamos codificación de variables categoricas
dataset = pd.get_dummies(dataset, columns=["Type 1", "Type 2"])

In [26]:
# Seleccionamos y escalamos
X = dataset.drop("Legendary", axis=1)
y = dataset["Legendary"].to_numpy() # Transformamos a numpy para usar reshape

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

""" Dividimos el dataset """
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3) # 30% para Test, 70% para Train

""" Convertimos a tensores """
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

""" Carga de los datos e DataLoader"""
train = TensorDataset(X_train_tensor, y_train_tensor) # Juntamos todo el dataset de entrenamiento
train_loader = DataLoader(train, batch_size=32, shuffle=True) 

test  = TensorDataset(X_test_tensor, y_test_tensor) # Juntamos todo el dataset de testeo
test_loader  = DataLoader(test, batch_size=32)

# OBS: utilizamos batch_size=32 para mantener un equilibrio 
# (al ser mas o menos pequeños, es más rápido pero a la vez mantiene estabilidad)

In [61]:
print("Legendarios en entrenamiento:", y_train_tensor.sum().item())
print("No legendarios:", len(y_train_tensor) - y_train_tensor.sum().item())


Legendarios en entrenamiento: 40.0
No legendarios: 520.0


### Implementación de la red neuronal
En el siguiente bloque implemente la arquitectura de red neuronal que predecirá si un Pokémon es legendario o no.

In [6]:
import torch.nn as nn
import torch.optim as optim # contiene algoritmos de optimización como SGD o Adam
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [55]:
# Red neuronal: 2 entradas → 4 neuronas ocultas → 1 salida
input_dim = X_train_tensor.shape[1] # 44
hidden_dim = 88
output_dim = 1

model = nn.Sequential(
    nn.Linear(input_dim, hidden_dim), # capa oculta 
    nn.ReLU(),                        # función de activación
    nn.Linear(hidden_dim, 64),        # capa oculta
    nn.ReLU(),
    nn.Linear(64, output_dim),        # salida con 1 neurona
    nn.Sigmoid()                      # función de activación de salida
)

In [27]:
print(X_train_tensor.shape[1])

44


En el siguiente bloque cree una instancia de la red neuronal y defina la función de pérdida y el optimizador. Justifique la elección de la función de pérdida y el optimizador.

In [56]:
""" Función de perdida """
criterion = nn.BCELoss()  # (Binary Cross-Entropy Loss)
# Justificación: Usada para clasificación binaria, el cual es este caso (legendario - no legendario)

""" Optimizador """
optimizer = optim.SGD(model.parameters(), lr=0.001)
# Justificación: Pensamos que podría ser una buena opción para el dataset
# este actualiza los parametros dando pequeños pasos en la dirección que reduce el error
# Se decidió utilizar un learning rate (lr) pequeño para dar pasos mas estables (aunque más lentos)

En el siguiente bloque entrene la red neuronal.

In [64]:

num_epochs = 200

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Época {epoch}, pérdida: {loss.item():.4f}")

Época 0, pérdida: 0.0358
Época 10, pérdida: 0.2035
Época 20, pérdida: 0.0595
Época 30, pérdida: 0.2331
Época 40, pérdida: 0.1497
Época 50, pérdida: 0.2341
Época 60, pérdida: 0.0337
Época 70, pérdida: 0.1188
Época 80, pérdida: 0.0287
Época 90, pérdida: 0.0281
Época 100, pérdida: 0.1637
Época 110, pérdida: 0.1522
Época 120, pérdida: 0.0845
Época 130, pérdida: 0.0964
Época 140, pérdida: 0.0952
Época 150, pérdida: 0.1853
Época 160, pérdida: 0.0778
Época 170, pérdida: 0.1681
Época 180, pérdida: 0.2079
Época 190, pérdida: 0.1992


Se realizó una prueba, para ver más o menos en cuantas epocas convergía:
(más o menos parece ocurrir entre la epoca 120 y 200)
```
Época 0, pérdida: 0.3458  
Época 10, pérdida: 0.0642  
Época 20, pérdida: 0.0693  
Época 30, pérdida: 0.4736  
Época 40, pérdida: 0.1876  
Época 50, pérdida: 0.0619  
Época 60, pérdida: 0.1785  
Época 70, pérdida: 0.0570  
Época 80, pérdida: 0.0565  
Época 90, pérdida: 0.0813  
Época 100, pérdida: 0.1554  
Época 110, pérdida: 0.2659  
Época 120, pérdida: 0.0454  
Época 130, pérdida: 0.0548  
Época 140, pérdida: 0.2213  
Época 150, pérdida: 0.1240  
Época 160, pérdida: 0.0244  
Época 170, pérdida: 0.0410  
Época 180, pérdida: 0.1791  
Época 190, pérdida: 0.1218  
Época 200, pérdida: 0.1544  
Época 210, pérdida: 0.0539  
Época 220, pérdida: 0.0707  
Época 230, pérdida: 0.1222  
Época 240, pérdida: 0.1902  
Época 250, pérdida: 0.0901  
Época 260, pérdida: 0.2040  
Época 270, pérdida: 0.0270  
Época 280, pérdida: 0.1860  
Época 290, pérdida: 0.1301  
Época 300, pérdida: 0.1201  
Época 310, pérdida: 0.1437  
Época 320, pérdida: 0.0828  
Época 330, pérdida: 0.1994  
Época 340, pérdida: 0.1987  
Época 350, pérdida: 0.0232  
Época 360, pérdida: 0.3386  
Época 370, pérdida: 0.1644  
Época 380, pérdida: 0.1918  
Época 390, pérdida: 0.1566  
Época 400, pérdida: 0.0879  
Época 410, pérdida: 0.1948  
Época 420, pérdida: 0.2751  
Época 430, pérdida: 0.1777  
Época 440, pérdida: 0.0876  
Época 450, pérdida: 0.0379  
Época 460, pérdida: 0.0487  
Época 470, pérdida: 0.1748  
Época 480, pérdida: 0.0857  
Época 490, pérdida: 0.1173  
```


En el siguiente bloque evalue el rendimiento de la red (Métricas de evaluación, matriz de confusión, etc.) y responda las siguientes preguntas:
- ¿Cuántas épocas fueron necesarias para converger?
- ¿Cuál fue la precisión final?
- ¿El entrenamiento de su red neuronal fue exitoso?
- ¿Cuales son los ejemplos que más confunde a la red neuronal? Responda el porqué de acuerdo a su conocimiento del dataset.



In [66]:
# probamos con test
with torch.no_grad(): # Desactivamos el cálculo de la gradiente
    y_test_pred = model(X_test_tensor) # Hace una predicción sobre los datos de prueba
    y_test_pred_labels = (y_test_pred > 0.5).float() # Convierte la probabilidad en una clasificación binaria
    acc = (y_test_pred_labels == y_test_tensor).float().mean() # Compara las predicciones con las etiquetas reales
    print(f"\nPrecisión en test: {acc.item():.2f}")



Precisión en test: 0.92


In [67]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Convertimos tensores a NumPy
y_true = y_test_tensor.numpy()
y_pred = y_test_pred_labels.numpy()

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
print("\nMatriz de confusión:")
print(cm)

# Precisión y recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")


Matriz de confusión:
[[213   2]
 [ 18   7]]
Precisión: 0.78
Recall: 0.28


In [68]:
print("Cantidad de legendarios reales:", y_test_tensor.sum().item())
print("Cantidad de legendarios predichos:", y_test_pred_labels.sum().item())

Cantidad de legendarios reales: 25.0
Cantidad de legendarios predichos: 9.0


### Conclusiones
En el siguiente bloque escriba sus conclusiones sobre el trabajo realizado. Además responda las siguientes preguntas:
- Sobre la matriz de confusión, si usted fuese un entrenador Pokémon, ¿Elegiría a Pokémon ubicados en FP o FN para su equipo? ¿Por qué?
- Entrene un modelo de aprendizaje automático diferente al de la red neuronal (eg. RF) y compare los resultados. ¿Cuál fue el mejor modelo? ¿Por qué?

## Problema 2

### Representaciones internas a partir de datos sin etiquetar  
En esta sección se trabajará con un enfoque diferente al utilizado previamente. Ahora no se utilizarán etiquetas para guiar el aprendizaje del modelo. En su lugar, el objetivo será encontrar una forma eficiente de representar cada instancia del dataset en un espacio de menor dimensión.

Diseñe un modelo que reduzca la dimensionalidad de los datos, y entrene dicho modelo utilizando el conjunto completo. Luego, visualice la representación de los datos en dos dimensiones. Para el resto siga un esquema similar al del problema anterior.

Responda las siguientes preguntas:
- ¿Se observan agrupaciones o patrones en la representación obtenida?
- ¿Es posible distinguir algún tipo de separación visual al colorear los puntos según si un Pokémon es legendario o no?
- ¿Qué interpretación podrías darle a la distribución resultante?
- ¿Sus resultados tienen alguna relación con el problema anterior?